# Installation

In [1]:
# Install condacolab
!pip install -q condacolab

# Import and install Conda using condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:06
🔁 Restarting kernel...


In [1]:
# Install Pyomo and solvers from the conda-forge channel
!conda install -c conda-forge pyomo ipopt coin-or-bonmin coincbc glpk -y > /dev/null 2>&1

# Install IDAES
!pip install -q idaes-pse --pre > /dev/null 2>&1

# Install the IDAES solver binaries, including Couenne
!idaes get-extensions --to /usr/local/bin > /dev/null 2>&1

In [2]:
import shutil

# Verify Pyomo installation
pyomo_path = shutil.which("pyomo")
print(f"Pyomo path: {pyomo_path}")

# Verify IPOPT installation
ipopt_path = shutil.which("ipopt")
print(f"IPOPT path: {ipopt_path}")

# Verify Bonmin installation
bonmin_path = shutil.which("bonmin")
print(f"Bonmin path: {bonmin_path}")

# Verify CBC installation
cbc_path = shutil.which("cbc")
print(f"CBC path: {cbc_path}")

# Verify GLPK installation
glpk_path = shutil.which("glpsol")
print(f"GLPK path: {glpk_path}")

# Verify Couenne installation
couenne_path = shutil.which("couenne")
print(f"Couenne path: {couenne_path}")

Pyomo path: /usr/local/bin/pyomo
IPOPT path: /usr/local/bin/ipopt
Bonmin path: /usr/local/bin/bonmin
CBC path: /usr/local/bin/cbc
GLPK path: /usr/local/bin/glpsol
Couenne path: /usr/local/bin/couenne


# Prepare the Data

In [5]:
!pip install -q scikit-learn statsmodels

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
import statsmodels.api as sm
import pyomo.environ as pyo

In [7]:
# Generate a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# SKLearn & StatsModels

In [8]:
# Logistic Regression with scikit-learn
sklearn_model = LogisticRegression(solver='lbfgs')
sklearn_model.fit(X_train, y_train)

y_pred_sklearn = sklearn_model.predict(X_test)
y_pred_proba_sklearn = sklearn_model.predict_proba(X_test)[:, 1]

accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
log_loss_sklearn = log_loss(y_test, y_pred_proba_sklearn)
print(f"Accuracy (scikit-learn): {accuracy_sklearn:.4f}")
print(f"Log Loss (scikit-learn): {log_loss_sklearn:.4f}")

Accuracy (scikit-learn): 0.8467
Log Loss (scikit-learn): 0.3566


In [9]:
# Logistic Regression with statsmodels
X_train_sm = sm.add_constant(X_train)  # Adding intercept term
X_test_sm = sm.add_constant(X_test)

statsmodels_model = sm.Logit(y_train, X_train_sm).fit()

y_pred_proba_sm = statsmodels_model.predict(X_test_sm)
y_pred_sm = (y_pred_proba_sm >= 0.5).astype(int)

accuracy_sm = accuracy_score(y_test, y_pred_sm)
log_loss_sm = log_loss(y_test, y_pred_proba_sm)
print(f"\nAccuracy (statsmodels): {accuracy_sm:.4f}")
print(f"Log Loss (statsmodels): {log_loss_sm:.4f}")

Optimization terminated successfully.
         Current function value: 0.322850
         Iterations 8

Accuracy (statsmodels): 0.8400
Log Loss (statsmodels): 0.3576


# Pyomo Solution

In [10]:
class LogisticRegressionPyomo:
    """
    Logistic Regression model using Pyomo for Maximum Likelihood Estimation (MLE).

    Attributes:
    -----------
    X : np.ndarray
        The feature matrix.
    y : np.ndarray
        The target vector.
    beta : np.ndarray
        The estimated coefficients after fitting the model.
    model : pyomo.ConcreteModel
        The Pyomo optimization model.

    Methods:
    --------
    fit(solver_name='ipopt'):
        Fits the logistic regression model using the selected Pyomo solver.
    predict_proba(X_new):
        Predicts probabilities for new data.
    predict(X_new):
        Predicts binary class labels for new data.
    evaluate(X_test, y_test):
        Evaluates the model on test data, returning accuracy and log loss.
    """

    def __init__(self, X, y):
        """
        Initializes the LogisticRegressionPyomo model with the provided data.

        Parameters:
        -----------
        X : np.ndarray
            The feature matrix.
        y : np.ndarray
            The target vector.
        """
        self.X = X
        self.y = y
        self.model = None
        self.beta = None

    def fit(self, solver_name='ipopt'):
        """
        Fits the logistic regression model using Pyomo for Maximum Likelihood Estimation (MLE).
        The user can choose the solver.

        Parameters:
        -----------
        solver_name : str, optional (default='ipopt')
            The name of the solver to use (e.g., 'ipopt', 'cbc', 'glpk').

        Returns:
        --------
        beta : np.ndarray
            The estimated coefficients.
        """
        n_samples, n_features = self.X.shape

        # Create a Pyomo model
        self.model = pyo.ConcreteModel()

        # Define variables (beta coefficients) for each feature
        self.model.beta = pyo.Var(range(n_features), domain=pyo.Reals, initialize=0.0)

        # Objective: Negative Log-Likelihood
        def neg_log_likelihood_rule(m):
            neg_log_likelihood = 0
            for i in range(n_samples):
                z = sum(m.beta[j] * self.X[i, j] for j in range(n_features))
                p = 1 / (1 + pyo.exp(-z))  # Sigmoid function using Pyomo's exp
                epsilon = 1e-8  # To prevent log(0)
                neg_log_likelihood += -self.y[i] * pyo.log(p + epsilon) - (1 - self.y[i]) * pyo.log(1 - p + epsilon)
            return neg_log_likelihood

        # Minimize the negative log-likelihood
        self.model.obj = pyo.Objective(rule=neg_log_likelihood_rule, sense=pyo.minimize)

        # Use the user-specified solver
        solver = pyo.SolverFactory(solver_name)

        # Check if the solver is available
        if not solver.available():
            raise ValueError(f"Solver '{solver_name}' is not available. Please ensure it is installed.")

        # Solve the optimization problem
        result = solver.solve(self.model)

        # Store the estimated coefficients
        self.beta = np.array([pyo.value(self.model.beta[j]) for j in range(n_features)])

        return self.beta

    def predict_proba(self, X_new):
        """
        Predicts probabilities for new data.

        Parameters:
        -----------
        X_new : np.ndarray
            The new feature matrix.

        Returns:
        --------
        np.ndarray
            The predicted probabilities for each instance in X_new.
        """
        z = np.dot(X_new, self.beta)
        return 1 / (1 + np.exp(-z))  # Sigmoid using NumPy for prediction

    def predict(self, X_new):
        """
        Predicts binary class labels for new data.

        Parameters:
        -----------
        X_new : np.ndarray
            The new feature matrix.

        Returns:
        --------
        np.ndarray
            The predicted binary class labels.
        """
        return (self.predict_proba(X_new) >= 0.5).astype(int)

    def evaluate(self, X_test, y_test):
        """
        Evaluates the model on test data, returning accuracy and log loss.

        Parameters:
        -----------
        X_test : np.ndarray
            The test feature matrix.
        y_test : np.ndarray
            The test target vector.

        Returns:
        --------
        tuple
            A tuple containing the accuracy and log loss on the test data.
        """
        y_pred_proba = self.predict_proba(X_test)
        y_pred = self.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        log_loss_val = log_loss(y_test, y_pred_proba)
        return accuracy, log_loss_val

In [11]:
# Initialize the Logistic Regression model using Pyomo
logreg_pyomo = LogisticRegressionPyomo(X_train, y_train)

# Fit the model using IPOPT solver (default)
beta_ipopt = logreg_pyomo.fit(solver_name='ipopt')
print(f"Estimated Coefficients (IPOPT):\n{beta_ipopt}")

# Evaluate the model
accuracy_ipopt, log_loss_ipopt = logreg_pyomo.evaluate(X_test, y_test)
print(f"\nAccuracy (IPOPT): {accuracy_ipopt:.4f}")
print(f"Log Loss (IPOPT): {log_loss_ipopt:.4f}")

Estimated Coefficients (IPOPT):
[-0.50881399  0.11611761 -1.19623329  0.08706355 -0.07761979 -0.29737023
  1.685782   -0.0079884  -1.09529722  0.06244599]

Accuracy (IPOPT): 0.8400
Log Loss (IPOPT): 0.3557


In [12]:
# Initialize the Logistic Regression model using Pyomo
logreg_pyomo = LogisticRegressionPyomo(X_train, y_train)

# Fit the model using Bonmin solver
beta_bonmin = logreg_pyomo.fit(solver_name='bonmin')
print(f"Estimated Coefficients (Bonmin):\n{beta_bonmin}")

# Evaluate the model
accuracy_bonmin, log_loss_bonmin = logreg_pyomo.evaluate(X_test, y_test)
print(f"\nAccuracy (Bonmin): {accuracy_bonmin:.4f}")
print(f"Log Loss (Bonmin): {log_loss_bonmin:.4f}")

Estimated Coefficients (Bonmin):
[-0.50881399  0.11611761 -1.19623329  0.08706355 -0.07761979 -0.29737023
  1.685782   -0.0079884  -1.09529722  0.06244599]

Accuracy (Bonmin): 0.8400
Log Loss (Bonmin): 0.3557


# PyQUBO Solution

In [13]:
!pip install -q pyqubo dimod

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.8 MB/s eta 0:00:00


In [14]:
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import dimod
from neal import SimulatedAnnealingSampler

In [15]:
class LogisticRegressionQUBO:
    """
    Logistic Regression model using QUBO formulation optimized with a Simulated Annealing Sampler.

    Attributes:
    -----------
    X : np.ndarray
        The feature matrix.
    y : np.ndarray
        The target vector.
    n_bits : int
        The number of bits used to represent each weight.

    Methods:
    --------
    fit(lam=10):
        Fits the logistic regression model by minimizing the QUBO objective.
    predict_proba(X_new):
        Predicts probabilities for new data.
    predict(X_new):
        Predicts binary class labels for new data.
    evaluate(X_test, y_test):
        Evaluates the model on test data, returning accuracy and log loss.
    """

    def __init__(self, X, y, n_bits=4):
        self.X = X
        self.y = y
        self.n_bits = n_bits
        self.beta = None  # Binary weights for the model

    def fit(self, lam=10):
        """
        Fits the logistic regression model by minimizing the QUBO objective using simulated annealing.

        Parameters:
        -----------
        lam : float, optional (default=10)
            Regularization parameter to control the tradeoff between fitting and constraint satisfaction.

        Returns:
        --------
        beta : np.ndarray
            The estimated coefficients.
        """
        n_samples, n_features = self.X.shape

        # Define binary weights for each feature
        weight_bits = Array.create('w', shape=(n_features, self.n_bits), vartype='BINARY')

        # QUBO objective for logistic regression
        qubo_objective = 0
        for i in range(n_samples):
            # Approximate z = X[i] . beta using binary weights
            z_i = sum((2**k) * self.X[i, j] * weight_bits[j, k] for j in range(n_features) for k in range(self.n_bits))

            # Logistic loss approximation (using a linear penalty instead of sigmoid)
            if self.y[i] == 1:
                qubo_objective += (1 - z_i) ** 2  # Penalty for positive class
            else:
                qubo_objective += (z_i) ** 2  # Penalty for negative class

        # Add constraints to ensure binary weights represent real-valued coefficients
        qubo_objective += Placeholder('lambda') * sum(
            (sum(weight_bits[j, k] for k in range(self.n_bits)) - 1) ** 2
            for j in range(n_features)
        )

        # Compile the QUBO model
        compiled_qubo = qubo_objective.compile()
        qubo, offset = compiled_qubo.to_qubo(feed_dict={'lambda': lam})

        # Use the Simulated Annealing Sampler to solve the QUBO problem
        sampler = SimulatedAnnealingSampler()
        response = sampler.sample_qubo(qubo, num_reads=100)
        best_solution = response.first.sample

        # Extract the binary weights from the solution
        self.beta = np.array([sum((2**k) * best_solution[f'w[{j}][{k}]'] for k in range(self.n_bits))
                              for j in range(n_features)])

        return self.beta

    def predict_proba(self, X_new):
        """
        Predicts probabilities for new data.

        Parameters:
        -----------
        X_new : np.ndarray
            The new feature matrix.

        Returns:
        --------
        np.ndarray
            The predicted probabilities for each instance in X_new.
        """
        z = np.dot(X_new, self.beta)
        return 1 / (1 + np.exp(-z))  # Sigmoid using NumPy for prediction

    def predict(self, X_new):
        """
        Predicts binary class labels for new data.

        Parameters:
        -----------
        X_new : np.ndarray
            The new feature matrix.

        Returns:
        --------
        np.ndarray
            The predicted binary class labels.
        """
        return (self.predict_proba(X_new) >= 0.5).astype(int)

    def evaluate(self, X_test, y_test):
        """
        Evaluates the model on test data, returning accuracy and log loss.

        Parameters:
        -----------
        X_test : np.ndarray
            The test feature matrix.
        y_test : np.ndarray
            The test target vector.

        Returns:
        --------
        tuple
            A tuple containing the accuracy and log loss on the test data.
        """
        y_pred_proba = self.predict_proba(X_test)
        y_pred = self.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        log_loss_val = log_loss(y_test, y_pred_proba)
        return accuracy, log_loss_val

In [16]:
# Initialize the Logistic Regression model using QUBO
logreg_qubo = LogisticRegressionQUBO(X_train, y_train, n_bits=4)

# Fit the model
beta_qubo = logreg_qubo.fit()
print(f"Estimated Coefficients (QUBO):\n{beta_qubo}")

# Evaluate the model
accuracy_qubo, log_loss_qubo = logreg_qubo.evaluate(X_test, y_test)
print(f"\nAccuracy (QUBO): {accuracy_qubo:.4f}")
print(f"Log Loss (QUBO): {log_loss_qubo:.4f}")

Estimated Coefficients (QUBO):
[1 0 1 0 0 0 2 0 1 0]

Accuracy (QUBO): 0.8500
Log Loss (QUBO): 0.5763
